In [5]:
import pandas as pd

# Load the Excel file
file_path = r"C:\Users\CHAKU FOODS\Documents\Somanya Farmers Geoboundaries.xlsx"

# Step 1: Load the data
sheet1 = pd.read_excel(file_path, sheet_name='Somanya Farmers Geoboundaries')
sheet2 = pd.read_excel(file_path, sheet_name='Names')

# Step 2: Define a function to process the "full_name" column
def process_full_name(row, name_reference):
    full_name = row['full_name']
    farm_number = None

    # Check if "Farm X" exists and extract it
    if "Farm" in full_name:
        parts = full_name.split("Farm")
        farm_number = f"Farm {parts[-1].strip()}"
        full_name = parts[0].strip()

    # Split remaining full_name into potential first and last names
    name_parts = full_name.split()
    if len(name_parts) > 1:
        # Infer last_name and first_name from the reference sheet
        last_name, first_name = name_parts[-1], " ".join(name_parts[:-1])
        # Validate against the reference sheet
        match = name_reference[
            (name_reference['last_name'] == last_name) & 
            (name_reference['first_name'].str.contains(first_name, na=False))
        ]
        if not match.empty:
            # Use the first matched name for consistency
            first_name = match.iloc[0]['first_name']
            last_name = match.iloc[0]['last_name']
    else:
        # If only one part, consider it as last_name, first_name unknown
        last_name = name_parts[0]
        first_name = None

    return pd.Series([last_name, first_name, farm_number])

# Step 3: Apply the function to process the data
name_reference = sheet2[['last_name', 'first_name']]
sheet1[['last_name', 'first_name', 'farm_number']] = sheet1.apply(
    process_full_name, axis=1, name_reference=name_reference
)

# Display the updated data
print(sheet1.head())

# Save the updated sheet1 to a new Excel file
output_path = r"C:\Users\CHAKU FOODS\Documents\Updated Names and Farms.xlsx"
sheet1.to_excel(output_path, index=False)


    Crop                full_name  \
0  Mango      Stephen Doku Farm 1   
1  Mango      Stephen Doku Farm 2   
2  Mango           Joseph  Tetteh   
3  Mango  Emmanuel Adjetey Annang   
4  Mango      Vida Ayoleh Antonio   

                              land_coordinates  latitude  longitude  altitude  \
0   6.1017683 0.0243283 58.099999999999994 1.9  6.101768   0.024328      58.1   
1                 6.1007293 0.0264156 62.7 3.5  6.100729   0.024713      62.7   
2                 6.1023905 0.0247133 57.0 4.2  6.102391   0.024713      57.0   
3                 6.0619631 0.0177471 86.9 4.7  6.061963   0.017747      86.9   
4  6.0745677 -0.0139678 97.69999999999999 3.65  6.074568  -0.013968      97.7   

   precision                              geographic_boundaries last_name  \
0       1.90  6.101746 0.0243237 61.400000000000006 2.4;6.10...      Doku   
1       3.50  6.1006581 0.026417 74.1 2.04;6.1006824 0.02643...      Doku   
2       4.20  6.1025726 0.024584 54.599999999999994 2.057;6